In [1]:
# !rm -r ieee
# !rm -r codes
# !git clone https://github.com/PiotrekGa/ieee.git
# !mv ieee/codes .

In [13]:
import os
import gc
import numpy as np
import pandas as pd
import joblib
from datetime import datetime

import lightgbm as lgb
import optuna

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, train_test_split

from codes import utils
from codes import fe_browser
from codes import fe_emails
from codes import fe_cards
from codes import fe_date
from codes import fe_relatives
from codes import fe_categorical
from codes import prepro
from codes import fe_users

SyntaxError: invalid syntax (<ipython-input-13-605e53a200f0>, line 8)

In [3]:
DATA_PATH = '../input/'
SEARCH_PARAMS = True

In [4]:
train, test, sample_submission = utils.import_data(DATA_PATH)

### Some Feature Engineering
train, test = fe_users.users_stats(train, test)

train, test = utils.drop_columns(train, test)

train, test = fe_browser.latest(train, test)

train, test = fe_emails.proton(train, test)

train['nulls1'] = train.isna().sum(axis=1)
test['nulls1'] = test.isna().sum(axis=1)

train, test = fe_emails.mappings(train, test)
train, test = fe_emails.labeling(train, test)

train, test = fe_cards.stats(train, test)

train, test = fe_relatives.divisions(train, test)

train, test = fe_date.dates(train, test)

train, test = fe_categorical.pairs(train, test)
train, test = fe_categorical.wtf(train, test)

start
hashing finished


/Users/piotrgabrys/data/python3/ieee/codes/fe_users.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  all_data = pd.concat([train, test])


concat data finished
grouping finished
stats calculated
merged all data
50 features are going to be dropped for being useless


In [5]:
train.shape

(590540, 436)

In [6]:
X_train = train.copy()
X_test = test.copy()

del train, test

#fill in mean for floats
X_train, X_test = prepro.prepro(X_train, X_test)

y_train = X_train['isFraud'].copy()
X_train = X_train.drop('isFraud', axis=1)

In [7]:
X_train = utils.reduce_mem_usage(X_train)

Mem. usage decreased to 566.00 Mb (26.0% reduction)


### Model and training

In [8]:
submission = sample_submission.copy()
submission['isFraud'] = 0

In [9]:
model = LGBMClassifier(metric='auc')

In [10]:
def cross_val_score2(model, X_train, y_train, n_fold):
    
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True)
    
    model_scores = []
    for train_index, valid_index in folds.split(X_train, y_train):
        X_train_, X_valid = X_train.iloc[train_index], X_train.iloc[valid_index]
        y_train_, y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]
        model.fit(X_train_,y_train_)
        del X_train_,y_train_
        val = model.predict_proba(X_valid)[:,1]
        del X_valid
        model_scores.append(roc_auc_score(y_valid, val))
        print('ROC accuracy: {}'.format(model_scores[-1]))
        del val, y_valid
    
    print('')
    return np.mean(model_scores)

In [11]:
def objective(trial):
    
    joblib.dump(study, 'study.pkl')
    
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, y_train, stratify=y_train)
    
    dtrain = lgb.Dataset(X_train_, label=y_train_)
    dtest = lgb.Dataset(X_test_, label=y_test_)

    param = {
        'objective': 'binary',
        'metric': 'binary_error',
        'verbosity': -1,
        'num_leaves': trial.suggest_int('num_leaves', 10, 500),
        'max_depth': trial.suggest_int('max_depth', 10, 300), 
        'n_estimators': trial.suggest_int('n_estimators', 50, 2000), 
        'subsample_for_bin': trial.suggest_int('subsample_for_bin', 1000, 500000), 
        'min_child_samples': trial.suggest_int('min_child_samples', 2, 200), 
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 0.00001, 10.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.0001, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.000001, 10.0)   
    }

    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'binary_error')
    gbm = lgb.train(
        param, dtrain, valid_sets=[dtest], verbose_eval=False, callbacks=[pruning_callback])

    preds = gbm.predict(X_test_)
    
    del X_train_, X_test_, y_train_
    
    gc.collect()
    
    return - roc_auc_score(y_test_, preds)

In [12]:
if SEARCH_PARAMS:
    study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=1))
    study.optimize(objective, timeout=60*60*6)

    joblib.dump(study, 'study.pkl')

    trials_df = pd.DataFrame([trial.value, trial.params] for trial in study.trials)
    trials_df.columns = ['value', 'params']
    trials_df.sort_values(by='value', inplace=True)
    params_for_cv = 10
    trials_df = trials_df.iloc[:params_for_cv,:]

    model = lgb.LGBMClassifier(metric='auc')
    n_folds = 8

    best_params = None
    best_value = 0

    for params in trials_df.params:
        model.set_params(**params)
        score = cross_val_score2(model, X_train, y_train, n_folds)

        if score > best_value:
            best_value = score
            best_params = params

        gc.collect()
        
else:
    best_params = {'num_leaves': 302,
                   'max_depth': 157,
                   'n_estimators': 1200,
                   'subsample_for_bin': 290858,
                   'min_child_samples': 79,
                   'reg_alpha': 1.0919573524807885,
                   'colsample_bytree': 0.5653288564015742,
                   'learning_rate': 0.028565794309535042}

W0815 22:23:00.895629 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-11-fcb5bab59cc2>", line 5, in objective
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, y_train, stratify=y_train)
NameError: name 'train_test_split' is not defined
W0815 22:23:00.899963 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-11-fcb5bab59cc2>", line 5, in objective
  

NameError: name 'train_test_split' is not defined
W0815 22:23:00.993427 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-11-fcb5bab59cc2>", line 5, in objective
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, y_train, stratify=y_train)
NameError: name 'train_test_split' is not defined
W0815 22:23:01.002568 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipytho

NameError: name 'train_test_split' is not defined
W0815 22:23:01.130659 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-11-fcb5bab59cc2>", line 5, in objective
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, y_train, stratify=y_train)
NameError: name 'train_test_split' is not defined
W0815 22:23:01.138394 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipytho

NameError: name 'train_test_split' is not defined
W0815 22:23:01.303682 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-11-fcb5bab59cc2>", line 5, in objective
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, y_train, stratify=y_train)
NameError: name 'train_test_split' is not defined
W0815 22:23:01.318229 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipytho

NameError: name 'train_test_split' is not defined
W0815 22:23:01.468997 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-11-fcb5bab59cc2>", line 5, in objective
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, y_train, stratify=y_train)
NameError: name 'train_test_split' is not defined
W0815 22:23:01.480941 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipytho

...

NameError: name 'train_test_split' is not defined
W0815 22:23:14.897809 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-11-fcb5bab59cc2>", line 5, in objective
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, y_train, stratify=y_train)
NameError: name 'train_test_split' is not defined
W0815 22:23:14.944186 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipytho

NameError: name 'train_test_split' is not defined
W0815 22:23:15.714411 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-11-fcb5bab59cc2>", line 5, in objective
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, y_train, stratify=y_train)
NameError: name 'train_test_split' is not defined
W0815 22:23:15.762378 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipytho

NameError: name 'train_test_split' is not defined
W0815 22:23:16.462601 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-11-fcb5bab59cc2>", line 5, in objective
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, y_train, stratify=y_train)
NameError: name 'train_test_split' is not defined
W0815 22:23:16.511950 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipytho

NameError: name 'train_test_split' is not defined
W0815 22:23:17.168122 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipython-input-11-fcb5bab59cc2>", line 5, in objective
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, y_train, stratify=y_train)
NameError: name 'train_test_split' is not defined
W0815 22:23:17.221403 4677830080 study.py:416] Setting trial status as TrialState.FAIL because of the following error: NameError("name 'train_test_split' is not defined",)
Traceback (most recent call last):
  File "/Users/piotrgabrys/.pyenv/versions/miniconda3-4.3.30/lib/python3.6/site-packages/optuna/study.py", line 407, in _run_trial
    result = func(trial)
  File "<ipytho

KeyboardInterrupt: 

In [ ]:
n_fold = 8
folds = StratifiedKFold(n_splits=n_fold, shuffle=True)

model = lgb.LGBMClassifier(metric='auc')
model.set_params(**best_params)

model_scores = []
model_scores_tr = []
for train_index, valid_index in folds.split(X_train, y_train):
    
    X_train_, X_valid = X_train.iloc[train_index], X_train.iloc[valid_index]
    y_train_, y_valid = y_train.iloc[train_index], y_train.iloc[valid_index]
    model.fit(X_train_,y_train_)
    train_val = model.predict_proba(X_train_)[:,1]
    model_scores_tr.append(roc_auc_score(y_train_, train_val))
    del X_train_,y_train_, train_val
    pred = model.predict_proba(X_test)[:,1]
    val = model.predict_proba(X_valid)[:,1]
    
    del X_valid
    model_scores.append(roc_auc_score(y_valid, val))
    print('ROC accuracy: {}, ROC train: {}'.format(model_scores[-1], model_scores_tr[-1]))
    del val, y_valid
    submission['isFraud'] = submission['isFraud'] + pred / n_fold
    del pred
    gc.collect()
    
model_score = np.round(np.mean(model_scores),4)
print(model_score)
print(params)

In [ ]:
timestamp = str(int(datetime.timestamp(datetime.now())))
submission.to_csv('{}_submission_{}.csv'.format(timestamp, str(model_score)))